In [1]:
'''
A notebook to create mapping and insert documents in elasticsearch(v 6.1)
'''

'\nA notebook to create mapping and insert documents in elasticsearch(v 6.1)\n'

In [2]:
import json

from os.path import join

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [3]:
ES_HOST = 'http://localhost'
ES_PORT = 9202
ES_HOST_STRING = '{}:{}'.format(ES_HOST, ES_PORT)

INDEX_NAME = 'movies_v1'
TYPE_NAME = 'marvel'

In [4]:
mapping = '''{
  "mappings": {
    "marvel": {
      "properties": {
        "title": {
          "type": "text"
        },
        "year": {
          "type": "long"
        }
      }
    }
  }
}'''

In [5]:
es = Elasticsearch([ES_HOST_STRING])

In [6]:
es.indices.create(index=INDEX_NAME, ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}

In [9]:
# lets insert documents now
data_file = join('data', 'marvel_movies.json')

In [10]:
with open(data_file) as infile:
    data = json.load(infile)

In [11]:
def make_documents(doc, doc_id):
    es_doc = {
        '_op_type': 'index',
        '_index': INDEX_NAME,
        '_type': TYPE_NAME,
        '_id': doc_id,
        '_source': doc
    }
    return es_doc

In [13]:
bulk(es, [make_documents(doc, idx) for idx, doc in enumerate(data)], index=INDEX_NAME, doc_type=TYPE_NAME)

(22, [])

In [20]:
count = es.count(index=INDEX_NAME, doc_type=TYPE_NAME, body={ "query": {"match_all" : { }}}).get('count')

In [25]:
assert len(data) == count